# Tratamento e geração do csv

In [159]:
!pip install googlemaps
!pip install odfpy

In [160]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import googlemaps
from datetime import datetime
from google.colab import userdata

# # Código de tratamento
# for ano in range(2015, 2020):
#   if f'Acidentes {ano}.ods' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.ods

# for ano in range(2020, 2025):
#   if f'Acidentes {ano}.xls' not in os.listdir():
#     !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/Acidentes%20{ano}.xls

# dados = pd.DataFrame()

# for ano in range (2015, 2020):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.ods', engine='odf')])

# dados1 = dados.copy()

# for ano in range(2020, 2025):
#   dados = pd.concat([dados, pd.read_excel(f'Acidentes {ano}.xls')])

# df = dados.sort_values(by=['DATA', 'HORA'])

# df['CRUZAMENTO'] = df['CRUZAMENTO'].astype(str) + df['CRUZAMENTO.1'].astype(str)
# df['LOGRADOURO'] = df['LOGRADOURO'].astype(str) + df['LOGRADOURO1'].astype(str)
# df = df.drop(columns=['LOGRADOURO1', 'CRUZAMENTO.1', 'Unnamed: 0', 2])
# df.rename(columns={'Nº': 'NUMERO'}, inplace=True)
# df.to_csv('acidentes.csv', index=False)

# def remove_nans(s):
#     s = str(s)
#     if s == 'nan':
#         return s
#     elif s.startswith("nan"):
#         return s[3:]  # Skip the first 3 characters if they are "nan"
#     elif s.endswith("nan"):
#         return s[:-3]  # Skip the last 3 characters if they are "nan"
#     else:
#       return s

# df['LOGRADOURO'] = df['LOGRADOURO'].apply(remove_nans)
# df['CRUZAMENTO'] = df['CRUZAMENTO'].apply(remove_nans)

# df['DATA'] = pd.to_datetime(df['DATA'])
# df['NUMERO'] = df['NUMERO'].astype(str)
# df['lat'] = ''
# df['lng'] = ''
# df['types'] = ''
# df['bairro'] = ''
# df.rename(columns={'DATA': 'data', 'HORA': 'hora', 'TEMPO': 'tempo',
#                    'TIPO_ACIDENTE': 'tipo_acidente', 'GRAVIDADE': 'gravidade',
#                    'LOGRADOURO': 'logradouro', 'NUMERO': 'numero', 'CRUZAMENTO': 'cruzamento'}, inplace=True)

In [162]:
# gravidades = list(df['gravidade'].unique())

# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if pd.isna(x) else x)
# df['gravidade'] = df['gravidade'].apply(lambda x: 'nan' if x == ' ' else x)

,data,hora,tempo,tipo_acidente,gravidade,logradouro,numero,cruzamento,lat,lng,types,bairro
0,2021-01-01,07:14:00,BOM,CHOQUE,nan,Avenida Almirante Saldanha da Gama,0,S/ LESÃO,,,,
1,2021-01-01,10:35:00,CHUVA,CHOQUE,nan,Rua Ana Santos,0,S/ LESÃOAvenida Nossa Senhora de Fátima,,,,
2,2021-01-01,12:45:00,BOM,ABALROAMENTO,nan,Rua Boris Kauffmann,0,C/ VÍTIMAS LEVESAvenida Nossa Senhora de Fátima,,,,
3,2021-01-02,01:15:00,BOM,COLISÃO,nan,Avenida Bartholomeu de Gusmão,0,S/ LESÃOAvenida Coronel Joaquim Montenegro,,,,
4,2021-01-02,03:14:00,BOM,TOMBAMENTO,nan,Avenida Doutor Waldemar Leão,0,C/ VÍTIMAS LEVESAvenida Rangel Pestana,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
2060,2021-12-31,01:25:00,BOM,CHOQUE,nan,Avenida General Francisco Glycerio,0,S/ LESÃOPraça Nenê Ferreira Martins,,,,
2061,2021-12-31,06:28:00,BOM,CHOQUE,nan,Rua Dom Gaspar de Afonseca,0,S/ LESÃORua Tocantins,,,,
2062,2021-12-31,18:44:00,CHUVA,CHOQUE,nan,Avenida Prefeito Doutor Antônio Manoel de Carv...,0,S/ LESÃO,,,,
2063,2021-12-31,23:55:00,BOM,COLISÃO,nan,Rua Monteiro Lobato,0,S/ LESÃOAvenida Presidente Wilson,,,,


In [164]:
# gravidades = list(df['gravidade'].unique())
# gravidades.remove('nan')
# df['temp'] = df['cruzamento']

# new_gravidade_values = []
# new_temp_values = []

# for index, row in df.iterrows():
#     temp_value = row['temp']
#     for gravidade in gravidades:
#         if str(gravidade) in temp_value:
#             new_gravidade_values.append(str(gravidade))
#             temp_value = temp_value.replace(str(gravidade), '')
#             break
#     else:
#         new_gravidade_values.append(np.nan)

#     new_temp_values.append(temp_value)

# df['temp'] = new_temp_values

# df['cruzamento'] = df['temp']
# df.drop(columns=['temp'], axis=1, inplace=True)

In [ ]:
# df[df['data'].dt.year == 2021]
# df['cruzamento'] = df['cruzamento'].apply(lambda x: 'nan' if x == '' else x)
# df['cruzamento'].unique()

In [ ]:
chave = userdata.get('chave')

gmaps = googlemaps.Client(key=chave)

In [167]:
for index, row in df.iterrows():
  if row['cruzamento'] == 'nan':
    local = row['numero'] + ' ' + row['logradouro']
  else:
    local = row['logradouro'] + ' ' + row['cruzamento']

  #print(local)
  geocode_result = gmaps.geocode(f'{local}, Santos, SP, Brazil')

  if row['cruzamento'] == 'nan':
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][2]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'
  else:
    if len(geocode_result[0]['address_components']) > 2 and 'long_name' in geocode_result[0]['address_components'][2]:
      df.at[index, 'bairro'] = geocode_result[0]['address_components'][1]['long_name']
    else:
      print(index)
      df.at[index, 'bairro'] = 'Bairro não encontrado'

  df.at[index, 'types'] = geocode_result[0]['types']
  df.at[index, 'lat'] = geocode_result[0]['geometry']['location']['lat']
  df.at[index, 'lng'] = geocode_result[0]['geometry']['location']['lng']

NameError: name 'gmaps' is not defined

# Análise

In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np

if 'acidentes.csv' not in os.listdir():
  !wget https://raw.githubusercontent.com/namespaiva/pi-acidentes/main/dados/acidentes.csv
  df = pd.read_csv('acidentes.csv')



In [168]:
df.to_csv('acidentesupdate.csv', index=False)